# 📘 Titration Curves: Acid-Base Equilibria> Determine pKa from titration data⏱️ **15-25 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Chemistry**---

## 🔬 Domain Background**Model:** $pH = pK_a + \log\frac{[A^-]}{[HA]}$**Applications:** pKa determination, buffer analysis---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
import numpy as npimport jax.numpy as jnpfrom nlsq import curve_fitimport matplotlib.pyplot as pltdef titration_curve(V, pKa, V_eq):    fraction = V / V_eq    fraction = jnp.clip(fraction, 1e-10, 1-1e-10)    return pKa + jnp.log10(fraction / (1 - fraction))

## Generate Data

In [ ]:
pKa_true, V_eq_true = 4.76, 25.0V = np.linspace(0.1, 49, 50)pH_true = titration_curve(V, pKa_true, V_eq_true)pH = pH_true + np.random.normal(0, 0.1, len(V))plt.plot(V, pH, 'o')plt.xlabel('Volume NaOH (mL)')plt.ylabel('pH')plt.tight_layout()
display(fig)
plt.close(fig)


## Fit Model

In [ ]:
mask = (V > 5) & (V < 45)popt, pcov = curve_fit(titration_curve, V[mask], pH[mask], p0=[4.5, 24])pKa_fit, V_eq_fit = poptprint(f'pKa: {pKa_fit:.2f}')print(f'Equivalence point: {V_eq_fit:.1f} mL')

## Key Insights1. **pKa at half-equivalence point**2. **Buffer capacity maximum at pH = pKa**---